- # Japan API

In [2]:
from urllib.request import urlopen
import json
import pandas as pd
import datetime as dt
import pandas_read_xml as pdx
from functools import reduce
from google.cloud import storage

- ## Extraction to dataframe

In [ ]:
def deep_get(dictionary, keys, default=None):
    return reduce(lambda d, key: d.get(key, default) if isinstance(d, dict) else default, keys.split("."), dictionary)

In [ ]:
df = pdx.read_xml('http://service.iris.edu/fdsnws/event/1/query?includeallmagnitudes=TRUE&magnitudetype=all&minlatitude=30.180889170292048&minlongitude=129.20644141355123&maxlatitude=45.540717058168504&maxlongitude=146.68542509079882&starttime=2021-01-01T00:00:00&endtime=2022-01-01T00:00:00&format=xml')

json_obj = df.to_json()

json_format = json.loads(json_obj)

data = pd.json_normalize(json_format)

df_events = data['q:quakeml.0.eventParameters.event']

events = []
for entry in df_events:
    for l in entry:
        events.append(l)

events_values = {'type' : [], 
                'place' : [], 
                'time' : [],
                'author' : [],
                'latitude' : [],
                'longitude' : [],
                'depth' : [],
                'mag' : [],
                'magType' : []}
                
for event in events:
    events_values['type'].append(deep_get(event, 'type')) 
    events_values['place'].append(deep_get(event, 'description.text')) 
    events_values['time'].append(deep_get(event, 'origin.time.value')) 
    events_values['author'].append(deep_get(event, 'origin.creationInfo.author')) 
    events_values['latitude'].append(deep_get(event, 'origin.latitude.value')) 
    events_values['longitude'].append(deep_get(event, 'origin.longitude.value')) 
    events_values['depth'].append(deep_get(event, 'origin.depth.value')) 
    events_values['mag'].append(deep_get(event, 'magnitude.mag.value')) 
    events_values['magType'].append(deep_get(event, 'magnitude.type'))

df_api_japan = pd.DataFrame(events_values)

- # Connection to cloud storage

In [6]:
current_time = str(dt.datetime.now().strftime('%Y_%m_%d_%H_%M_%S'))
current_time

'2022_11_23_11_45_09'

In [ ]:
client = storage.Client(project='Seismic Alert System')
bucket = client.get_bucket('seismic-data-bucket')
blob = bucket.blob(f'api_japan_{current_time}.csv')
blob.upload_from_string(df.to_csv(index = False),content_type = 'csv')

- # Defining the final extraction function

In [ ]:
def japan_api_to_gcs():
    def deep_get(dictionary, keys, default=None):
        return reduce(lambda d, key: d.get(key, default) if isinstance(d, dict) else default, keys.split("."), dictionary)
    
    df = pdx.read_xml('http://service.iris.edu/fdsnws/event/1/query?includeallmagnitudes=TRUE&magnitudetype=all&minlatitude=30.180889170292048&minlongitude=129.20644141355123&maxlatitude=45.540717058168504&maxlongitude=146.68542509079882&starttime=2021-01-01T00:00:00&endtime=2022-01-01T00:00:00&format=xml')

    json_obj = df.to_json()

    json_format = json.loads(json_obj)

    data = pd.json_normalize(json_format)

    df_events = data['q:quakeml.0.eventParameters.event']

    events = []
    for entry in df_events:
        for l in entry:
            events.append(l)

    events_values = {'type' : [], 
                    'place' : [], 
                    'time' : [],
                    'author' : [],
                    'latitude' : [],
                    'longitude' : [],
                    'depth' : [],
                    'mag' : [],
                    'magType' : []}
                    
    for event in events:
        events_values['type'].append(deep_get(event, 'type')) 
        events_values['place'].append(deep_get(event, 'description.text')) 
        events_values['time'].append(deep_get(event, 'origin.time.value')) 
        events_values['author'].append(deep_get(event, 'origin.creationInfo.author')) 
        events_values['latitude'].append(deep_get(event, 'origin.latitude.value')) 
        events_values['longitude'].append(deep_get(event, 'origin.longitude.value')) 
        events_values['depth'].append(deep_get(event, 'origin.depth.value')) 
        events_values['mag'].append(deep_get(event, 'magnitude.mag.value')) 
        events_values['magType'].append(deep_get(event, 'magnitude.type'))

    df_api_japan = pd.DataFrame(events_values)

    current_time = str(dt.datetime.now().strftime('%Y_%m_%d_%H_%M_%S'))

    client = storage.Client(project='Seismic Alert System')
    bucket = client.get_bucket('seismic-data-bucket')
    blob = bucket.blob("APIs/" + f'api_japan_{current_time}.csv')
    blob.upload_from_string(df_api_japan.to_csv(index = False),content_type = 'csv')

# -------------------------------------------------------Transformation------------------------------------------------------------------------------------------------

    df_api_japan_t = df_api_japan
    
    df_api_japan_t['time'] = pd.to_datetime(df_api_japan_t['time'])#, unit='ns')

    df_api_japan_t.latitude = df_api_japan_t.latitude.astype(float)
    df_api_japan_t.longitude = df_api_japan_t.longitude.astype(float)
    df_api_japan_t.depth = df_api_japan_t.depth.astype(float)
    df_api_japan_t.mag = df_api_japan_t.mag.astype(float)

    df_api_japan_t = df_api_japan_t[['time', 'place', 'mag', 'magType', 'depth', 'latitude', 'longitude']]

    df_api_japan_t.place = df_api_japan_t.place.str.title()

    no_japan_location = df_api_japan_t[df_api_japan.place.str.contains('Japan|Kuril|Honshu') == False].index
    df_api_japan_t.drop(no_japan_location, inplace=True)    

    client = storage.Client(project='Seismic Alert System')
    bucket = client.get_bucket('seismic-data-bucket')
    blob = bucket.blob("Transformed_Data/" + f'api_japan_{current_time}.csv')
    blob.upload_from_string(df_api_japan_t.to_csv(index = False),content_type = 'csv')


def chile_api_to_cgs():
    url = 'https://chilealerta.com/api/query/?user=demo&select=ultimos_sismos&limit=100&country=Chile'
    json_obj = urlopen(url)
    data = json.load(json_obj)
    df_api_chile = pd.json_normalize(data, record_path=['ultimos_sismos_Chile'])

    current_time = str(dt.datetime.now().strftime('%Y_%m_%d_%H_%M_%S'))

    client = storage.Client(project='Seismic Alert System')
    bucket = client.get_bucket('seismic-data-bucket')
    blob = bucket.blob("APIs/" + f'api_chile_{current_time}.csv')
    blob.upload_from_string(df_api_chile.to_csv(index = False),content_type = 'csv')

    #-----------------------------------Transformation----------------------------------------------------------------------

    df_api_chile_t = df_api_chile

    df_api_chile_t.drop(columns=['state', 'local_time', 'chilean_time', 'id', 'url', 'source'], inplace=True)

    df_api_chile_t.rename(columns={'utc_time' : 'time',
                             'reference' : 'place',
                             'magnitude' : 'mag',
                             'scale' : 'magType'}, inplace=True)

    df_api_chile_t.time = pd.to_datetime(df_api_chile_t.time)

    df_api_chile_t = df_api_chile_t[['time', 'place', 'mag', 'magType', 'depth', 'latitude', 'longitude']]

    client = storage.Client(project='Seismic Alert System')
    bucket = client.get_bucket('seismic-data-bucket')
    blob = bucket.blob("Transformed_Data/" + f'api_chile_{current_time}.csv')
    blob.upload_from_string(df_api_chile_t.to_csv(index = False),content_type = 'csv')

def us_api_to_cgs():
    url = 'https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_hour.geojson'
    json_obj = urlopen(url)
    data = json.load(json_obj)

    df_us_api = pd.json_normalize(data, record_path=['features'])

    current_time = str(dt.datetime.now().strftime('%Y_%m_%d_%H_%M_%S'))

    client = storage.Client(project='Seismic Alert System')
    bucket = client.get_bucket('seismic-data-bucket')
    blob = bucket.blob("APIs/" + f'api_us_{current_time}.csv')
    blob.upload_from_string(df_us_api.to_csv(index = False),content_type = 'csv')

    #-----------------------------------Transformation----------------------------------------------------------------------

    df_api_us_t = df_us_api[['properties.time', 'properties.place', 'properties.mag', 'properties.magType', 'geometry.coordinates']]

    df_api_us_t.rename(columns={'properties.time' : 'time',
                          'properties.place' : 'place',
                          'properties.mag' : 'mag',
                          'properties.magType' : 'magType',
                          'geometry.coordinates' : 'coordinates'}, inplace=True)

    df_us_locations = pd.DataFrame(df_api_us_t["coordinates"].to_list(), columns=['longitude', 'latitude', 'depth'])
    df_api_us_t['latitude'] = df_us_locations.latitude
    df_api_us_t['longitude'] = df_us_locations.longitude
    df_api_us_t['depth'] = df_us_locations.depth
    df_api_us_t.drop(columns='coordinates', inplace=True)

    df_api_us_t.time = pd.to_datetime(df_api_us_t.time, unit='ms')

    other_countries_locations = df_api_us_t[(((df_api_us_t.latitude < 20.911795455444313) & (df_api_us_t.latitude > 50.02924641916901)) & ((df_api_us_t.longitude < -124.65301770531302) & (df_api_us_t.longitude > -66.95789388605114))) | (df_api_us_t.place.str.contains('Japan|Russia|Canada|Mexico|Sea'))].index # I create the index refering to those entries
    df_api_us_t.drop(other_countries_locations, inplace=True)

    df_api_us_t = df_api_us_t[['time', 'place', 'mag', 'magType', 'depth', 'latitude', 'longitude']]

    client = storage.Client(project='Seismic Alert System')
    bucket = client.get_bucket('seismic-data-bucket')
    blob = bucket.blob("Transformed_Data/" + f'api_us_{current_time}.csv')
    blob.upload_from_string(df_api_us_t.to_csv(index = False),content_type = 'csv')

def main(data, context):
    japan_api_to_gcs()
    chile_api_to_cgs()
    us_api_to_cgs()

- # Defining the transformations function trying this

In [ ]:
def transformation_us_data():

    from google.cloud import storage
    import pandas as pd
    import io

    storage_client = storage.Client()

    source_bucket = storage_client.bucket(source_bucket)

    col_names = ['type','id','properties.mag','properties.place','properties.time','properties.updated','properties.tz','properties.url',
    'properties.detail','properties.felt','properties.cdi','properties.mmi','properties.alert','properties.status','properties.tsunami','properties.sig',
    'properties.net','properties.code','properties.ids','properties.sources','properties.types','properties.nst','properties.dmin','properties.rms',
    'properties.gap','properties.magType','properties.type','properties.title','geometry.type','geometry.coordinates']

    df = pd.DataFrame(columns=col_names)

    for file in list(source_bucket.list_blobs()):
        file_path="gs://{}/{}".format(file.bucket.name, file.name)
        df = df.append(pd.read_csv(file_path, header=None, names=col_names))

    df_api_us_t = df[['properties.time', 'properties.place', 'properties.mag', 'properties.magType', 'geometry.coordinates']]

    df_api_us_t.rename(columns={'properties.time' : 'time',
                          'properties.place' : 'place',
                          'properties.mag' : 'mag',
                          'properties.magType' : 'magType',
                          'geometry.coordinates' : 'coordinates'}, inplace=True)

    df_us_locations = pd.DataFrame(df_api_us_t["coordinates"].to_list(), columns=['longitude', 'latitude', 'depth'])
    df_api_us_t['latitude'] = df_us_locations.latitude
    df_api_us_t['longitude'] = df_us_locations.longitude
    df_api_us_t['depth'] = df_us_locations.depth
    df_api_us_t.drop(columns='coordinates', inplace=True)

    df_api_us_t.time = pd.to_datetime(df_api_us_t.time, unit='ms')

    other_countries_locations = df_api_us_t[(((df_api_us_t.latitude < 20.911795455444313) & (df_api_us_t.latitude > 50.02924641916901)) & ((df_api_us_t.longitude < -124.65301770531302) & (df_api_us_t.longitude > -66.95789388605114))) | (df_api_us_t.place.str.contains('Japan|Russia|Canada|Mexico|Sea'))].index # I create the index refering to those entries
    df_api_us_t.drop(other_countries_locations, inplace=True)

    df_api_us_t = df_api_us_t[['time', 'place', 'mag', 'magType', 'depth', 'latitude', 'longitude']]

    client = storage.Client(project='Seismic Alert System')
    bucket = client.get_bucket('seismic-data-bucket')
    blob = bucket.blob("Transformed_Data/" + f'api_us_{current_time}.csv')
    blob.upload_from_string(df_api_us_t.to_csv(index = False),content_type = 'csv')

In [ ]:
def transformation_us_data():

    import subprocess

    result = subprocess.run(['gsutil', 'ls', 'seismic-data-bucket/US_API/*.csv'], stdout=subprocess.PIPE)

    all_dat = pd.DataFrame()
    for file in result.stdout.splitlines():
        dat = pd.read_csv(file.strip())
        all_dat = all_dat.append(dat, ignore_index=True)

    df_api_us_t = all_dat[['properties.time', 'properties.place', 'properties.mag', 'properties.magType', 'geometry.coordinates']]

    df_api_us_t.rename(columns={'properties.time' : 'time',
                          'properties.place' : 'place',
                          'properties.mag' : 'mag',
                          'properties.magType' : 'magType',
                          'geometry.coordinates' : 'coordinates'}, inplace=True)

    df_us_locations = pd.DataFrame(df_api_us_t["coordinates"].to_list(), columns=['longitude', 'latitude', 'depth'])
    df_api_us_t['latitude'] = df_us_locations.latitude
    df_api_us_t['longitude'] = df_us_locations.longitude
    df_api_us_t['depth'] = df_us_locations.depth
    df_api_us_t.drop(columns='coordinates', inplace=True)

    df_api_us_t.time = pd.to_datetime(df_api_us_t.time, unit='ms')

    other_countries_locations = df_api_us_t[(((df_api_us_t.latitude < 20.911795455444313) & (df_api_us_t.latitude > 50.02924641916901)) & ((df_api_us_t.longitude < -124.65301770531302) & (df_api_us_t.longitude > -66.95789388605114))) | (df_api_us_t.place.str.contains('Japan|Russia|Canada|Mexico|Sea'))].index # I create the index refering to those entries
    df_api_us_t.drop(other_countries_locations, inplace=True)

    df_api_us_t = df_api_us_t[['time', 'place', 'mag', 'magType', 'depth', 'latitude', 'longitude']]

    client = storage.Client(project='Seismic Alert System')
    bucket = client.get_bucket('seismic-data-bucket')
    blob = bucket.blob("Transformed_Data/" + f'api_us_{current_time}.csv')
    blob.upload_from_string(df_api_us_t.to_csv(index = False),content_type = 'csv')

In [ ]:
def transformation_us_data():
    
    storage_client = storage.Client()

    bucket = storage_client.bucket('seismic-data-bucket')
    blobs = bucket.list_blobs(prefix='US_API/')

    list_temp_raw = []
    for file in blobs:
        filename = file.name
        temp = pd.read_csv('gs://' + 'seismic-data-bucket' + '/' + filename, encoding='utf-8')
    list_temp_raw.append(temp)

    df = pd.concat(list_temp_raw)

    df_api_us_t = df[['properties.time', 'properties.place', 'properties.mag', 'properties.magType', 'geometry.coordinates']]

    df_api_us_t.rename(columns={'properties.time' : 'time',
                          'properties.place' : 'place',
                          'properties.mag' : 'mag',
                          'properties.magType' : 'magType',
                          'geometry.coordinates' : 'coordinates'}, inplace=True)

    df_us_locations = pd.DataFrame(df_api_us_t["coordinates"].to_list(), columns=['longitude', 'latitude', 'depth'])
    df_api_us_t['latitude'] = df_us_locations.latitude
    df_api_us_t['longitude'] = df_us_locations.longitude
    df_api_us_t['depth'] = df_us_locations.depth
    df_api_us_t.drop(columns='coordinates', inplace=True)

    df_api_us_t.time = pd.to_datetime(df_api_us_t.time, unit='ms')

    other_countries_locations = df_api_us_t[(((df_api_us_t.latitude < 20.911795455444313) & (df_api_us_t.latitude > 50.02924641916901)) & ((df_api_us_t.longitude < -124.65301770531302) & (df_api_us_t.longitude > -66.95789388605114))) | (df_api_us_t.place.str.contains('Japan|Russia|Canada|Mexico|Sea'))].index # I create the index refering to those entries
    df_api_us_t.drop(other_countries_locations, inplace=True)

    df_api_us_t = df_api_us_t[['time', 'place', 'mag', 'magType', 'depth', 'latitude', 'longitude']]

    client = storage.Client(project='Seismic Alert System')
    bucket = client.get_bucket('seismic-data-bucket')
    blob = bucket.blob("Transformed_Data/" + f'api_us_{current_time}.csv')
    blob.upload_from_string(df_api_us_t.to_csv(index = False),content_type = 'csv')



In [5]:
def us_data_transformation():
    
    df_api_us_t = df_api_us[['properties.time', 'properties.place', 'properties.mag', 'properties.magType', 'geometry.coordinates']]

    df_api_us_t.rename(columns={'properties.time' : 'time',
                          'properties.place' : 'place',
                          'properties.mag' : 'mag',
                          'properties.magType' : 'magType',
                          'geometry.coordinates' : 'coordinates'}, inplace=True)

    df_us_locations = pd.DataFrame(df_api_us_t["coordinates"].to_list(), columns=['longitude', 'latitude', 'depth'])
    df_api_us_t['latitude'] = df_us_locations.latitude
    df_api_us_t['longitude'] = df_us_locations.longitude
    df_api_us_t['depth'] = df_us_locations.depth
    df_api_us_t.drop(columns='coordinates', inplace=True)

    df_api_us_t.time = pd.to_datetime(df_api_us_t.time, unit='ms')

    other_countries_locations = df_api_us_t[(((df_api_us_t.latitude < 20.911795455444313) & (df_api_us_t.latitude > 50.02924641916901)) & ((df_api_us_t.longitude < -124.65301770531302) & (df_api_us_t.longitude > -66.95789388605114))) | (df_api_us_t.place.str.contains('Japan|Russia|Canada|Mexico|Sea'))].index # I create the index refering to those entries
    df_api_us_t.drop(other_countries_locations, inplace=True)

    df_api_us_t = df_api_us_t[['time', 'place', 'mag', 'magType', 'depth', 'latitude', 'longitude']]

def japan_data_transformation():

    df_api_japan_t = df_api_japan
    
    df_api_japan_t['time'] = pd.to_datetime(df_api_japan_t['time'])#, unit='ns')

    df_api_japan_t.latitude = df_api_japan_t.latitude.astype(float)
    df_api_japan_t.longitude = df_api_japan_t.longitude.astype(float)
    df_api_japan_t.depth = df_api_japan_t.depth.astype(float)
    df_api_japan_t.mag = df_api_japan_t.mag.astype(float)

    df_api_japan_t = df_api_japan_t[['time', 'place', 'mag', 'magType', 'depth', 'latitude', 'longitude']]

    df_api_japan_t.place = df_api_japan_t.place.str.title()

    no_japan_location = df_api_japan_t[df_api_japan.place.str.contains('Japan|Kuril|Honshu') == False].index
    df_api_japan_t.drop(no_japan_location, inplace=True)


def chile_data_transformation():
    df_api_chile_t = df_api_chile

    df_api_chile_t.drop(columns=['state', 'local_time', 'chilean_time', 'id', 'url', 'source'], inplace=True)

    df_api_chile_t.rename(columns={'utc_time' : 'time',
                             'reference' : 'place',
                             'magnitude' : 'mag',
                             'scale' : 'magType'}, inplace=True)

    df_api_chile_t.time = pd.to_datetime(df_api_chile_t.time)

    df_api_chile_t = df_api_chile_t[['time', 'place', 'mag', 'magType', 'depth', 'latitude', 'longitude']]


def main(data, context):
    us_data_transformation()
    japan_data_transformation()
    chile_data_transformation()

In [4]:
data = pd.read_csv('gs://seismic-data-bucket/APIs/api_japan_2022_11_23_16_02_14.csv')

_request non-retriable exception: Anonymous caller does not have storage.objects.get access to the Google Cloud Storage object. Permission 'storage.objects.get' denied on resource (or it may not exist)., 401
Traceback (most recent call last):
  File "c:\Users\Auli\AppData\Local\Programs\Python\Python310\lib\site-packages\gcsfs\retry.py", line 115, in retry_request
    return await func(*args, **kwargs)
  File "c:\Users\Auli\AppData\Local\Programs\Python\Python310\lib\site-packages\gcsfs\core.py", line 415, in _request
    validate_response(status, contents, path, args)
  File "c:\Users\Auli\AppData\Local\Programs\Python\Python310\lib\site-packages\gcsfs\retry.py", line 102, in validate_response
    raise HttpError(error)
gcsfs.retry.HttpError: Anonymous caller does not have storage.objects.get access to the Google Cloud Storage object. Permission 'storage.objects.get' denied on resource (or it may not exist)., 401


HttpError: Anonymous caller does not have storage.objects.get access to the Google Cloud Storage object. Permission 'storage.objects.get' denied on resource (or it may not exist)., 401

google function backup

In [ ]:
import json
import pandas as pd
import datetime as dt
import pandas_read_xml as pdx
from functools import reduce
from google.cloud import storage
from urllib.request import urlopen

def japan_api_to_gcs():
    def deep_get(dictionary, keys, default=None):
        return reduce(lambda d, key: d.get(key, default) if isinstance(d, dict) else default, keys.split("."), dictionary)
    
    df = pdx.read_xml('http://service.iris.edu/fdsnws/event/1/query?includeallmagnitudes=TRUE&magnitudetype=all&minlatitude=30.180889170292048&minlongitude=129.20644141355123&maxlatitude=45.540717058168504&maxlongitude=146.68542509079882&starttime=2021-01-01T00:00:00&endtime=2022-01-01T00:00:00&format=xml')

    json_obj = df.to_json()

    json_format = json.loads(json_obj)

    data = pd.json_normalize(json_format)

    df_events = data['q:quakeml.0.eventParameters.event']

    events = []
    for entry in df_events:
        for l in entry:
            events.append(l)

    events_values = {'type' : [], 
                    'place' : [], 
                    'time' : [],
                    'author' : [],
                    'latitude' : [],
                    'longitude' : [],
                    'depth' : [],
                    'mag' : [],
                    'magType' : []}
                    
    for event in events:
        events_values['type'].append(deep_get(event, 'type')) 
        events_values['place'].append(deep_get(event, 'description.text')) 
        events_values['time'].append(deep_get(event, 'origin.time.value')) 
        events_values['author'].append(deep_get(event, 'origin.creationInfo.author')) 
        events_values['latitude'].append(deep_get(event, 'origin.latitude.value')) 
        events_values['longitude'].append(deep_get(event, 'origin.longitude.value')) 
        events_values['depth'].append(deep_get(event, 'origin.depth.value')) 
        events_values['mag'].append(deep_get(event, 'magnitude.mag.value')) 
        events_values['magType'].append(deep_get(event, 'magnitude.type'))

    df_api_japan = pd.DataFrame(events_values)

    current_time = str(dt.datetime.now().strftime('%Y_%m_%d_%H_%M_%S'))

    client = storage.Client(project='Seismic Alert System')
    bucket = client.get_bucket('seismic-data-bucket')
    blob = bucket.blob("APIs/" + f'api_japan_{current_time}.csv')
    blob.upload_from_string(df_api_japan.to_csv(index = False),content_type = 'csv')

def chile_api_to_cgs():
    url = 'https://chilealerta.com/api/query/?user=demo&select=ultimos_sismos&limit=100&country=Chile'
    json_obj = urlopen(url)
    data = json.load(json_obj)
    df_api_chile = pd.json_normalize(data, record_path=['ultimos_sismos_Chile'])

    current_time = str(dt.datetime.now().strftime('%Y_%m_%d_%H_%M_%S'))

    client = storage.Client(project='Seismic Alert System')
    bucket = client.get_bucket('seismic-data-bucket')
    blob = bucket.blob("APIs/" + f'api_chile_{current_time}.csv')
    blob.upload_from_string(df_api_chile.to_csv(index = False),content_type = 'csv')

def us_api_to_cgs():
    url = 'https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_hour.geojson'
    json_obj = urlopen(url)
    data = json.load(json_obj)

    df_us_api = pd.json_normalize(data, record_path=['features'])

    current_time = str(dt.datetime.now().strftime('%Y_%m_%d_%H_%M_%S'))

    client = storage.Client(project='Seismic Alert System')
    bucket = client.get_bucket('seismic-data-bucket')
    blob = bucket.blob("US_API/" + f'api_us_{current_time}.csv')
    blob.upload_from_string(df_us_api.to_csv(index = False),content_type = 'csv')



def main(data, context):
    japan_api_to_gcs()
    chile_api_to_cgs()
    us_api_to_cgs()

In [12]:
data = pd.read_csv('apis datasets/api_us_20221123_111700.csv')
data2 = pd.read_csv('apis datasets/api_japan_20221123_111700.csv')

In [13]:
data

,time,place,mag,magType,depth,latitude,longitude
0,2021-12-31 14:31:34.776,"Southeast Of Honshu, Japan",4.3,mb,10000.0,31.1148,142.2999
1,2021-12-31 14:23:50.014,"Kyushu, Japan",4.0,mb,27260.0,32.1049,130.0111
2,2021-12-30 23:22:38.756,"Southeast Of Honshu, Japan",4.2,mb,10000.0,30.9772,142.3701
3,2021-12-30 19:19:16.109,"Off East Coast Of Honshu, Japan",4.5,mb,10000.0,33.1948,142.5877
4,2021-12-30 06:08:44.798,"Southeast Of Honshu, Japan",4.3,mb,10000.0,30.9180,141.8623
...,...,...,...,...,...,...,...
700,2021-01-05 21:02:41.764,"Southeast Of Honshu, Japan",4.1,mb,92210.0,33.0845,140.1500
701,2021-01-05 02:16:21.501,"Southeast Of Honshu, Japan",4.5,mb,35000.0,32.3940,141.2047
702,2021-01-03 14:14:54.692,Eastern Sea Of Japan,4.1,mb,250430.0,43.6306,138.7964
703,2021-01-02 00:45:00.272,"Off East Coast Of Honshu, Japan",4.6,mb,10000.0,33.4333,141.4589


In [14]:
data2

,time,place,mag,magType,depth,latitude,longitude
0,2021-12-31 14:31:34.776,"Southeast Of Honshu, Japan",4.3,mb,10000.0,31.1148,142.2999
1,2021-12-31 14:23:50.014,"Kyushu, Japan",4.0,mb,27260.0,32.1049,130.0111
2,2021-12-30 23:22:38.756,"Southeast Of Honshu, Japan",4.2,mb,10000.0,30.9772,142.3701
3,2021-12-30 19:19:16.109,"Off East Coast Of Honshu, Japan",4.5,mb,10000.0,33.1948,142.5877
4,2021-12-30 06:08:44.798,"Southeast Of Honshu, Japan",4.3,mb,10000.0,30.9180,141.8623
...,...,...,...,...,...,...,...
700,2021-01-05 21:02:41.764,"Southeast Of Honshu, Japan",4.1,mb,92210.0,33.0845,140.1500
701,2021-01-05 02:16:21.501,"Southeast Of Honshu, Japan",4.5,mb,35000.0,32.3940,141.2047
702,2021-01-03 14:14:54.692,Eastern Sea Of Japan,4.1,mb,250430.0,43.6306,138.7964
703,2021-01-02 00:45:00.272,"Off East Coast Of Honshu, Japan",4.6,mb,10000.0,33.4333,141.4589
